<a href="https://colab.research.google.com/github/hyeonwooCH/Final_physiognomy_palmistry/blob/main/mole_alg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 코랩 셀에 이 코드 붙이기
import os
import cv2
from PIL import Image, ImageOps
import subprocess

# 원본 이미지 경로
input_image_path = "/content/hyunj.jpg"

# EXIF 회전 처리된 이미지 저장할 임시 경로
fixed_image_path = "/content/h_temp.jpg"

# 1단계: EXIF 처리
img = Image.open(input_image_path)
img = ImageOps.exif_transpose(img)  # ★ 회전 적용
img.save(fixed_image_path)

print("✅ EXIF 처리 + 추론 완료!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. SegFace 저장소 클론 및 이동
!git clone https://github.com/Kartik-3004/SegFace.git
%cd SegFace

# 2. 필수 라이브러리 설치 (코랩 환경 최적화 버전)
!pip install -q timm==0.9.12 segmentation-models-pytorch albumentations python-dotenv huggingface_hub

In [ ]:
import os
from huggingface_hub import hf_hub_download

# 1. .env 경로 설정 파일 생성
root_path = "/content/SegFace"
with open(".env", "w") as f:
    f.write(f"ROOT_PATH={root_path}\n")
    f.write(f"DATA_PATH={root_path}/data\n")
    f.write(f"LOG_PATH={root_path}/logs\n")

# 2. 가중치(Weights) 다운로드
hf_hub_download(repo_id="kartiknarayan/SegFace",
                filename="convnext_celeba_512/model_299.pt",
                local_dir="./weights")

print("✅ 환경 설정 및 가중치 다운로드 완료!")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# 프로젝트 모듈 임포트 경로 추가
if '/content/SegFace' not in sys.path:
    sys.path.append('/content/SegFace')
from network.models.segface_celeb import SegFaceCeleb

# --- [설정 영역] ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/SegFace/weights/convnext_celeba_512/model_299.pt"
input_res = 512

# 테스트할 데이터 경로 (본인의 경로로 수정하세요)
# 예: /content/drive/MyDrive/test_data/images
test_images_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test/images"
test_masks_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test/masks"

# 수정된 class_names (SegFace 모델 출력 순서)
class_names = [
    'bg', 'neck', 'face', 'cloth', 'lr', 'rr', 'lb', 'rb', 'le',
    're', 'nose', 'imouth', 'llip', 'ulip', 'hair',
    'glass', 'hat', 'earr', 'neckl'
]
# ------------------



AI_HUB_TO_SEGFACE =  {
    0: 0, 1: 2, 2: 7, 3: 6, 4: 9, 5: 8, 6: 15, 7: 5, 8: 4, 9: 17,
    10: 10, 11: 11, 12: 13, 13: 12, 14: 1, 15: 18, 16: 3, 17: 14, 18: 16,
}

# 1. 모델 로드 함수
def load_segface_model():
    model = SegFaceCeleb(input_res, "convnext_base")
    checkpoint = torch.load(model_path, map_location=device)
    pretrained_dict = checkpoint.get('state_dict_backbone', checkpoint.get('model_state_dict', checkpoint))
    model.load_state_dict(pretrained_dict, strict=False)
    model.to(device)
    model.eval()
    return model

# 2. 다중 데이터 로더 클래스
class SegFaceEvalDataset(Dataset):
    def __init__(self, img_dir, mask_dir, res=512):
        import glob
        import os
        self.img_paths = sorted(glob.glob(os.path.join(img_dir, "*.*")))
        self.mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.*")))
        self.res = res
    def __len__(self): return len(self.img_paths)

    def align_aihub_labels(self, mask_np):
        """AI 허브 ID를 SegFace ID로 재배열"""
        new_mask = np.zeros_like(mask_np)
        for ai_id, seg_id in AI_HUB_TO_SEGFACE.items():
            new_mask[mask_np == ai_id] = seg_id
        return new_mask

    def __getitem__(self, idx):
        # 1. 이미지 로드
        img = Image.open(self.img_paths[idx]).convert('RGB').resize((self.res, self.res))
        img_tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1).float().div(255)

        # 2. 마스크 로드 (AI 허브 Grayscale 대응)
        mask = Image.open(self.mask_paths[idx]).convert('L')
        mask = mask.resize((self.res, self.res), Image.NEAREST)
        mask_np = np.array(mask).astype(np.int64)

        # 💡 [핵심] AI 허브 데이터 전처리
        # 만약 AI 허브 데이터가 0, 1, 2... 순서가 아니라
        # 시각화를 위해 큰 값(예: 10, 20...)을 가지고 있다면 정규화가 필요합니다.
        # 고유값을 확인한 후 아래 주석을 해제하여 사용하세요.
        mask_np = self.align_aihub_labels(mask_np)

        mask_tensor = torch.from_numpy(mask_np).long()
        return img_tensor, mask_tensor, os.path.basename(self.img_paths[idx])



In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class FaceParsingDataset(Dataset):
    def __init__(self, root_path, mode='test'):
        self.img_dir = os.path.join(root_path, mode, 'images')
        self.mask_dir = os.path.join(root_path, mode, 'masks')
        # 이미지 파일 목록 로드
        self.file_list = [f for f in sorted(os.listdir(self.img_dir)) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]

        # 파일명에서 확장자를 제외한 이름 추출 (예: '0174' -> '0174_grayscale.png')
        base_name = os.path.splitext(img_name)[0]
        mask_name = f"{base_name}_grayscale.png"

        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)

        # 이미지 및 마스크 로드
        img = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path)

        # 텐서 변환: [C, H, W] 형태 및 0~1 정규화
        img_tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1).float() / 255.0
        mask_np = np.array(mask)

        return img_tensor, mask_np, img_name

# 경로 설정 (실제 경로로 수정 필요)
BASE_PATH = '/content/drive/MyDrive/c. Final_Team/Split_dataset'
dataset = FaceParsingDataset(BASE_PATH, mode='test')
print(f"Dataset 로드 완료: {len(dataset)} 개의 파일을 찾았습니다.")

# 1. 모델 로드 (작성하신 load_segface_model 함수 호출)
model = load_segface_model()

# 2. 장치 설정 확인 (위 설정 영역에 있지만 다시 한 번 확실히)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ 모델 로드 완료! 장치: {device}")

In [ ]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image
from torchvision import transforms

# --- [1. SegFace 추론 함수 정의] ---
def process_new_input(image_path, model, device, res=512):
    orig_img = Image.open(image_path).convert('RGB')
    img_resized = orig_img.resize((res, res))
    img_tensor = torch.from_numpy(np.array(img_resized)).permute(2, 0, 1).float().div(255).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        # SegFace 특유의 forward 인자 대응 (None 전달)
        output = model(img_tensor, labels=None, dataset=None)
        if isinstance(output, (list, tuple)):
            output = output[0]
        pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

    return img_tensor.squeeze(0), pred_mask

# --- [2. 피부 점 탐지 함수 정의] ---
def detect_skin_moles(img_tensor, pred_mask):
    # 텐서를 Numpy로 변환
    img_np = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    # 2번 라벨(얼굴 피부)만 추출
    face_area = (pred_mask == 2).astype(np.uint8)

    # 전처리: 가우시안 블러로 미세 노이즈 제거
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Blackhat 연산: 밝은 배경에서 어두운 점 강조
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    blackhat = cv2.morphologyEx(blurred, cv2.MORPH_BLACKHAT, kernel)

    # 얼굴 영역으로 제한
    masked_blackhat = cv2.bitwise_and(blackhat, blackhat, mask=face_area)

    # 임계값 처리 및 컨투어 검출
    _, mole_bin = cv2.threshold(masked_blackhat, 10, 255, cv2.THRESH_BINARY)
    mole_mask = np.zeros_like(gray)
    contours, _ = cv2.findContours(mole_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 2 < area < 120: # 점 크기 필터
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0: continue
            circularity = 4 * np.pi * (area / (perimeter * perimeter))
            if circularity > 0.6: # 원형도 필터
                cv2.drawContours(mole_mask, [cnt], -1, 255, -1)

    return img_np, mole_mask

# --- [3. 메인 실행부] ---
sample_image_path = '/content/h_temp.jpg'

if os.path.exists(sample_image_path):
    # model과 device는 이미 선언되어 있어야 합니다 (SegFace 로드 셀 확인)
    try:
        # 1단계: SegFace 추론
        img_tensor, p_mask = process_new_input(sample_image_path, model, device)

        # 2단계: 점 탐지
        img_raw, mole_result = detect_skin_moles(img_tensor, p_mask)

        # 3단계: 시각화
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(img_raw)
        plt.title("Original Face")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(img_raw)
        mole_cmap = mcolors.ListedColormap([(0,0,0,0), 'red'])
        plt.imshow(mole_result, cmap=mole_cmap, alpha=0.8)
        plt.title("Detected Moles (Red Dots)")
        plt.axis('off')
        plt.show()

        # 결과 분석
        num_moles = len(cv2.findContours(mole_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0])
        print(f"✅ 발견된 점/잡티 개수: {num_moles}개")

    except NameError as e:
        print(f"❌ 설정 에러: 모델(model)이 로드되지 않았습니다. {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {sample_image_path}")